In [ ]:
import requests
import gzip
import os
import json

# --- 설정 ---
# 다운로드할 날짜
TARGET_DATE = "2025-06-01"
# 데이터를 저장할 경로
OUTPUT_DIR = r"C:\Users\EL040\Desktop\MS_3rd-Project\Project\data"
# ---

# 저장할 폴더가 없으면 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"[{TARGET_DATE}] 데이터 다운로드를 시작합니다.")
print(f"저장 위치: {OUTPUT_DIR}")

# 0시부터 23시까지 시간대별로 다운로드
for hour in range(24):
    # 다운로드 URL 생성
    file_name = f"{TARGET_DATE}-{hour}.json.gz"
    url = f"https://data.gharchive.org/{file_name}"
    
    gz_path = os.path.join(OUTPUT_DIR, file_name)
    json_path = os.path.join(OUTPUT_DIR, file_name.replace(".gz", ""))

    try:
        # 1. 데이터 다운로드
        print(f"다운로드 중... {url}")
        response = requests.get(url, stream=True)
        response.raise_for_status() # 오류가 있으면 예외 발생
        
        with open(gz_path, 'wb') as f:
            f.write(response.content)
        
        # 2. 압축 해제
        print(f"압축 해제 중... {gz_path}")
        with gzip.open(gz_path, 'rb') as f_in:
            with open(json_path, 'wb') as f_out:
                f_out.write(f_in.read())
                
        # 3. 원본 .gz 파일 삭제
        os.remove(gz_path)
        print(f"완료: {json_path}")

    except requests.exceptions.HTTPError as e:
        print(f"오류: {url} 파일을 찾을 수 없습니다. (HTTP Status: {e.response.status_code})")
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}")

print("모든 작업이 완료되었습니다.")